# 04 - Test and Deploy Training Pipeline to Vertex Pipelines

The purpose of this notebook is to test, deploy, and run the `TFX` pipeline on `Vertex Pipelines`. The notebook covers the following tasks:
1. Run the tests locally.
2. Run the pipeline using `Vertex Pipelines`
3. Execute the pipeline deployment `CI/CD` steps using `Cloud Build`.

## Setup

### Import libraries

In [1]:
%load_ext autoreload
%autoreload 2

import os
import kfp
import tfx.v1 as tfx

print("Tensorflow Version:", tfx.__version__)
print("KFP Version:", kfp.__version__)

Tensorflow Version: 1.8.0
KFP Version: 1.8.12


### Setup Google Cloud project

In [2]:
PROJECT = 'pbalm-cxb-aa'
REGION = 'europe-west4'
CF_REGION = 'europe-west1' # No Cloud Functions in europe-west4
BUCKET =  PROJECT + '-eu'
SERVICE_ACCOUNT = "188940921537-compute@developer.gserviceaccount.com"

if PROJECT == "" or PROJECT is None or PROJECT == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT = shell_output[0]
    
if SERVICE_ACCOUNT == "" or SERVICE_ACCOUNT is None or SERVICE_ACCOUNT == "[your-service-account]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.account)' 2>/dev/null
    SERVICE_ACCOUNT = shell_output[0]
    
if BUCKET == "" or BUCKET is None or BUCKET == "[your-bucket-name]":
    # Get your bucket name to GCP project id
    BUCKET = PROJECT
    # Try to create the bucket if it doesn't exists
    ! gsutil mb -l $REGION gs://$BUCKET
    print("")
    
print("Project ID:", PROJECT)
print("Region:", REGION)
print("Bucket name:", BUCKET)
print("Service Account:", SERVICE_ACCOUNT)

Project ID: pbalm-cxb-aa
Region: europe-west4
Bucket name: pbalm-cxb-aa-eu
Service Account: 188940921537-compute@developer.gserviceaccount.com


### Set configurations

In [3]:
BQ_LOCATION = 'EU'
BQ_DATASET_NAME = 'vertex_eu' # Change to your BQ dataset name.
BQ_TABLE_NAME = 'creditcards_ml'

VERSION = 'v02'
DATASET_DISPLAY_NAME = 'creditcards'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-train-pipeline'

CICD_IMAGE_NAME = 'cicd:latest'
CICD_IMAGE_URI = f"{REGION}-docker.pkg.dev/{PROJECT}/creditcards/{CICD_IMAGE_NAME}"

DATAFLOW_REGION = 'europe-west4'
DATAFLOW_SERVICE_ACCOUNT = SERVICE_ACCOUNT
DATAFLOW_SUBNETWORK = f'https://www.googleapis.com/compute/v1/projects/{PROJECT}/regions/{REGION}/subnetworks/default'


In [4]:
!rm -r src/raw_schema/.ipynb_checkpoints/

rm: cannot remove 'src/raw_schema/.ipynb_checkpoints/': No such file or directory


## 1. Run the CICD steps locally

### Set pipeline configurations for the local run

In [6]:
os.environ["DATASET_DISPLAY_NAME"] = DATASET_DISPLAY_NAME
os.environ["MODEL_DISPLAY_NAME"] =  MODEL_DISPLAY_NAME
os.environ["PIPELINE_NAME"] = PIPELINE_NAME
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["BQ_LOCATION"] = BQ_LOCATION
os.environ["BQ_DATASET_NAME"] = BQ_DATASET_NAME
os.environ["BQ_TABLE_NAME"] = BQ_TABLE_NAME
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/{DATASET_DISPLAY_NAME}/e2e_tests"
os.environ["TRAIN_LIMIT"] = "1000"
os.environ["TEST_LIMIT"] = "100"
os.environ["UPLOAD_MODEL"] = "0"
os.environ["ACCURACY_THRESHOLD"] = "-0.1"    # NB Negative accuracy threshold makes no sense - allows everything
os.environ["BEAM_RUNNER"] = "DirectRunner"
os.environ["TRAINING_RUNNER"] = "local"

In [7]:
from src.tfx_pipelines import config
import importlib
importlib.reload(config)

for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

PROJECT: pbalm-cxb-aa
REGION: europe-west4
GCS_LOCATION: gs://pbalm-cxb-aa-eu/creditcards/e2e_tests
ARTIFACT_STORE_URI: gs://pbalm-cxb-aa-eu/creditcards/e2e_tests/tfx_artifacts
MODEL_REGISTRY_URI: model_registry
DATASET_DISPLAY_NAME: creditcards
MODEL_DISPLAY_NAME: creditcards-classifier-v02
PIPELINE_NAME: creditcards-classifier-v02-train-pipeline
ML_USE_COLUMN: ml_use
EXCLUDE_COLUMNS: trip_start_timestamp
TRAIN_LIMIT: 1000
TEST_LIMIT: 100
SERVE_LIMIT: 0
NUM_TRAIN_SPLITS: 4
NUM_EVAL_SPLITS: 1
ACCURACY_THRESHOLD: -0.1
USE_KFP_SA: False
TFX_IMAGE_URI: gcr.io//tfx-chicago-taxi-tips:latest
BEAM_RUNNER: DirectRunner
SERVICE_ACCOUNT: 
SUBNETWORK: 
BEAM_DIRECT_PIPELINE_ARGS: ['--project=pbalm-cxb-aa', '--temp_location=gs://pbalm-cxb-aa-eu/creditcards/e2e_tests/temp']
BEAM_DATAFLOW_PIPELINE_ARGS: ['--project=pbalm-cxb-aa', '--temp_location=gs://pbalm-cxb-aa-eu/creditcards/e2e_tests/temp', '--region=europe-west4', '--runner=DirectRunner', '--service_account_email=', '--no_use_public_ips', '--su

### Run unit tests

In [ ]:
!py.test src/tests/datasource_utils_tests.py -s

In [ ]:
!py.test src/tests/model_tests.py -s

### Run e2e pipeline test

In [ ]:
!py.test src/tests/pipeline_deployment_tests.py::test_e2e_pipeline -s

## 2. Run the training pipeline using Vertex Pipelines



In [8]:
IMG_VERSION='tfx-1.8'

### Set the pipeline configurations for the Vertex AI run

In [9]:
os.environ["DATASET_DISPLAY_NAME"] = DATASET_DISPLAY_NAME
os.environ["MODEL_DISPLAY_NAME"] = MODEL_DISPLAY_NAME
os.environ["PIPELINE_NAME"] = PIPELINE_NAME
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = DATAFLOW_REGION
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/{DATASET_DISPLAY_NAME}"
os.environ["TRAIN_LIMIT"] = "85000"
os.environ["TEST_LIMIT"] = "15000"
os.environ["BEAM_RUNNER"] = "DataflowRunner"
os.environ["TRAINING_RUNNER"] = "vertex"
os.environ["TFX_IMAGE_URI"] = f"{REGION}-docker.pkg.dev/{PROJECT}/{DATASET_DISPLAY_NAME}/vertex:{IMG_VERSION}"
os.environ["ENABLE_CACHE"] = "1"
os.environ["SUBNETWORK"] = DATAFLOW_SUBNETWORK
os.environ["SERVICE_ACCOUNT"] = DATAFLOW_SERVICE_ACCOUNT

In [10]:
from src.tfx_pipelines import config
import importlib
importlib.reload(config)

for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

PROJECT: pbalm-cxb-aa
REGION: europe-west4
GCS_LOCATION: gs://pbalm-cxb-aa-eu/creditcards
ARTIFACT_STORE_URI: gs://pbalm-cxb-aa-eu/creditcards/tfx_artifacts
MODEL_REGISTRY_URI: model_registry
DATASET_DISPLAY_NAME: creditcards
MODEL_DISPLAY_NAME: creditcards-classifier-v02
PIPELINE_NAME: creditcards-classifier-v02-train-pipeline
ML_USE_COLUMN: ml_use
EXCLUDE_COLUMNS: trip_start_timestamp
TRAIN_LIMIT: 85000
TEST_LIMIT: 15000
SERVE_LIMIT: 0
NUM_TRAIN_SPLITS: 4
NUM_EVAL_SPLITS: 1
ACCURACY_THRESHOLD: -0.1
USE_KFP_SA: False
TFX_IMAGE_URI: europe-west4-docker.pkg.dev/pbalm-cxb-aa/creditcards/vertex:tfx-1.8
BEAM_RUNNER: DataflowRunner
SERVICE_ACCOUNT: 188940921537-compute@developer.gserviceaccount.com
SUBNETWORK: https://www.googleapis.com/compute/v1/projects/pbalm-cxb-aa/regions/europe-west4/subnetworks/default
BEAM_DIRECT_PIPELINE_ARGS: ['--project=pbalm-cxb-aa', '--temp_location=gs://pbalm-cxb-aa-eu/creditcards/temp']
BEAM_DATAFLOW_PIPELINE_ARGS: ['--project=pbalm-cxb-aa', '--temp_location=

### Build the ML container image

This is the `TFX` runtime environment for the training pipeline steps.

In [11]:
!echo $TFX_IMAGE_URI

europe-west4-docker.pkg.dev/pbalm-cxb-aa/creditcards/vertex:tfx-1.8


In [ ]:
!cp build/Dockerfile.vertex Dockerfile
!gcloud builds submit --tag $TFX_IMAGE_URI . --timeout=15m --machine-type=e2-highcpu-8

### Compile pipeline

In [12]:
from src.tfx_pipelines import runner

pipeline_definition_file = f'{config.PIPELINE_NAME}.json'
pipeline_definition = runner.compile_training_pipeline(pipeline_definition_file)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying etl.py -> build/lib
copying transformations.py -> build/lib
installing to /tmp/tmp8sl16ezl
running install
running install_lib
copying build/lib/transformations.py -> /tmp/tmp8sl16ezl
copying build/lib/etl.py -> /tmp/tmp8sl16ezl
running install_egg_info
running egg_info
creating tfx_user_code_DataTransformer.egg-info
writing tfx_user_code_DataTransformer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_DataTransformer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_DataTransformer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_DataTransformer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_DataTransformer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_DataTransformer.egg-info/SOURCES.txt'
Copying tfx_user_code_DataTransformer.egg-info to /tmp/tmp8sl16ezl/tfx_user_code_DataTransformer-0.0+29bc5439691a704c7e799c0c5bab8e8ad062a2

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying trainer.py -> build/lib
copying runner.py -> build/lib
copying model.py -> build/lib
copying defaults.py -> build/lib
copying exporter.py -> build/lib
copying data.py -> build/lib
copying task.py -> build/lib
installing to /tmp/tmpt6ayr9lg
running install
running install_lib
copying build/lib/trainer.py -> /tmp/tmpt6ayr9lg
copying build/lib/model.py -> /tmp/tmpt6ayr9lg
copying build/lib/runner.py -> /tmp/tmpt6ayr9lg
copying build/lib/task.py -> /tmp/tmpt6ayr9lg
copying build/lib/data.py -> /tmp/tmpt6ayr9lg
copying build/lib/defaults.py -> /tmp/tmpt6ayr9lg
copying build/lib/exporter.py -> /tmp/tmpt6ayr9lg
running install_egg_info
running egg_info
creating tfx_user_code_ModelTrainer.egg-info
writing tfx_user_code_ModelTrainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_ModelTrainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_ModelTrainer.egg-info/top_lev

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


In [13]:
from google.cloud import aiplatform as vertex_ai
vertex_ai.init(project=PROJECT, location=REGION)

In [14]:
PIPELINES_STORE = f"gs://{BUCKET}/{DATASET_DISPLAY_NAME}/compiled_pipelines/"
!gsutil cp {pipeline_definition_file} {PIPELINES_STORE}

Copying file://creditcards-classifier-v02-train-pipeline.json [Content-Type=application/json]...
/ [1 files][ 27.2 KiB/ 27.2 KiB]                                                
Operation completed over 1 objects/27.2 KiB.                                     


### Submit run to Vertex Pipelines

In [ ]:
from google.cloud.aiplatform import pipeline_jobs
    
job = pipeline_jobs.PipelineJob(template_path = pipeline_definition_file,
                                display_name=DATASET_DISPLAY_NAME,
                                parameter_values={
                                    'learning_rate': 0.003,
                                    'batch_size': 512,
                                    'hidden_units': '128,128',
                                    'num_epochs': 30,
                                })

job.run(sync=False, service_account=DATAFLOW_SERVICE_ACCOUNT)

### Extracting pipeline runs metadata

In [ ]:
from google.cloud import aiplatform as vertex_ai

pipeline_df = vertex_ai.get_pipeline_df(PIPELINE_NAME)
pipeline_df = pipeline_df[pipeline_df.pipeline_name == PIPELINE_NAME]
pipeline_df.T

## 3. Execute the pipeline deployment CI/CD steps in Cloud Build

The CI/CD routine is defined in the [pipeline-deployment.yaml](pipeline-deployment.yaml) file, and consists of the following steps:
1. Clone the repository to the build environment.
2. Run unit tests.
3. Run a local e2e test of the pipeline.
4. Build the ML container image for pipeline steps.
5. Compile the pipeline.
6. Upload the pipeline to Cloud Storage.

### Build CI/CD container Image for Cloud Build

This is the runtime environment where the steps of testing and deploying the pipeline will be executed.

In [ ]:
!echo $CICD_IMAGE_URI

In [ ]:
!gcloud builds submit --tag $CICD_IMAGE_URI build/. --timeout=15m --machine-type=e2-highcpu-8

### Run CI/CD from pipeline deployment using Cloud Build

In [ ]:
#REPO_URL = "https://github.com/GoogleCloudPlatform/mlops-with-vertex-ai.git" # Change to your github repo.
REPO_URL="https://github.com/pbalm/mlops-with-vertex-ai.git"

BRANCH = "main"

GCS_LOCATION = f"gs://{BUCKET}/{DATASET_DISPLAY_NAME}/"
TEST_GCS_LOCATION = f"gs://{BUCKET}/{DATASET_DISPLAY_NAME}/e2e_tests"
CI_TRAIN_LIMIT = 1000
CI_TEST_LIMIT = 100
CI_UPLOAD_MODEL = 0
CI_ACCURACY_THRESHOLD = -0.1 # again setting accuracy threshold to negative
BEAM_RUNNER = "DataflowRunner"
TRAINING_RUNNER = "vertex"
VERSION = 'tfx-1.8'
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-train-pipeline'
PIPELINES_STORE = os.path.join(GCS_LOCATION, "compiled_pipelines")

TFX_IMAGE_URI = f"gcr.io/{PROJECT}/{DATASET_DISPLAY_NAME}:{VERSION}"
#europe-west4-docker.pkg.dev/pbalm-cxb-aa/dataflow/creditcards@sha256:latest
TFX_IMAGE_URI = f"{REGION}-docker.pkg.dev/{PROJECT}/{DATASET_DISPLAY_NAME}/vertex:{VERSION}"

SUBSTITUTIONS=f"""\
_REPO_URL='{REPO_URL}',\
_BRANCH={BRANCH},\
_CICD_IMAGE_URI={CICD_IMAGE_URI},\
_PROJECT={PROJECT},\
_REGION={DATAFLOW_REGION},\
_GCS_LOCATION={GCS_LOCATION},\
_TEST_GCS_LOCATION={TEST_GCS_LOCATION},\
_BQ_LOCATION={BQ_LOCATION},\
_BQ_DATASET_NAME={BQ_DATASET_NAME},\
_BQ_TABLE_NAME={BQ_TABLE_NAME},\
_DATASET_DISPLAY_NAME={DATASET_DISPLAY_NAME},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_CI_TRAIN_LIMIT={CI_TRAIN_LIMIT},\
_CI_TEST_LIMIT={CI_TEST_LIMIT},\
_CI_UPLOAD_MODEL={CI_UPLOAD_MODEL},\
_CI_ACCURACY_THRESHOLD={CI_ACCURACY_THRESHOLD},\
_BEAM_RUNNER={BEAM_RUNNER},\
_TRAINING_RUNNER={TRAINING_RUNNER},\
_TFX_IMAGE_URI={TFX_IMAGE_URI},\
_PIPELINE_NAME={PIPELINE_NAME},\
_PIPELINES_STORE={PIPELINES_STORE},\
_SUBNETWORK={DATAFLOW_SUBNETWORK},\
_GCS_BUCKET={BUCKET}/cloudbuild,\
_SERVICE_ACCOUNT={DATAFLOW_SERVICE_ACCOUNT}\
"""
!echo $SUBSTITUTIONS

In [ ]:
!gcloud builds submit --no-source --timeout=60m --config build/pipeline-deployment.yaml --substitutions {SUBSTITUTIONS} --machine-type=e2-highcpu-8